# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [2]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
mlb_df=pd.read_csv("assets/mlb.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    
cities["NHL"] = cities["NHL"].apply(lambda x: re.sub(r"\[.+\]", "", x))
cities["NHL"] = cities["NHL"].replace({"RangersIslandersDevils": "Rangers,Islanders,Devils",
                                           "KingsDucks": "Kings,Ducks",
                                           "Red Wings": "Red,Wings", 
                                           "Maple Leafs": "Maple,Leafs", 
                                           "Blue Jackets": "Blue,Jackets",
                                           "Golden Knights": "Golden,Knights" })
cities["NHL"] = cities["NHL"].apply(lambda x: x.split(","))
cities = cities.explode("NHL")
cities["NBA"] = cities["NBA"].apply(lambda x: re.sub(r"\[.+\]", "", x))
cities["NBA"] = cities["NBA"].replace({"KnicksNets": "Knicks,Nets",
                                           "LakersClippers": "Lakers,Clippers",
                                           "Trail Blazers": "Trail,Blazers"})
cities["NBA"] = cities["NBA"].apply(lambda x: x.split(","))
cities = cities.explode("NBA")
cities["MLB"] = cities["MLB"].apply(lambda x: re.sub(r"\[.+\]", "", x))
cities["MLB"] = cities["MLB"].replace({"Blue Jays": "Blue,Jays", 
                                           "CubsWhite Sox": "Cubs,White,Sox", 
                                           "DodgersAngels": "Dodgers,Angels", 
                                           "GiantsAthletics": "Giants,Athletics", 
                                           "YankeesMets": "Yankees,Mets",
                                           "Red Sox": "Red,Sox"})
cities["MLB"] = cities["MLB"].apply(lambda x: x.split(","))
cities = cities.explode("MLB")
cities["NFL"] = cities["NFL"].apply(lambda x: re.sub(r"\[.+\]", "", x))
cities["NFL"] = cities["NFL"].replace({"GiantsJets": "Giants,Jets",
                                           "RamsChargers": "Rams,Chargers",
                                           "49ersRaiders": "49ers,Raiders"
                                           })
cities["NFL"] = cities["NFL"].apply(lambda x: x.split(","))
cities = cities.explode("NFL")
    
nhl_df = nhl_df[nhl_df["year"] == 2018]
nhl_df["team"] = nhl_df["team"].apply(lambda x: x.replace("*", ""))
nhl_df["team"] = nhl_df["team"].apply(lambda x: x.split(" ")[-1])
    
nhldf = pd.merge(cities, nhl_df, left_on="NHL", right_on="team")
nhldf = nhldf[["Metropolitan area", "Population (2016 est.)[8]", "NHL", "team", "W", "L"]]
nhldf["W-L%"] = nhldf["W"].astype("int")/(nhldf["W"].astype("int") + nhldf["L"].astype("int"))
nhldf["Population (2016 est.)[8]"] = nhldf["Population (2016 est.)[8]"].astype("float")
nhldf["W-L%"] = nhldf["W-L%"].astype("float")

nhldf.loc[nhldf["Metropolitan area"] == "New York City", "W-L%"] = 0.5182013333333334 # mean of NY W-L%
nhldf.loc[nhldf["Metropolitan area"] == "Los Angeles", "W-L%"] = 0.6228945 # mean of LA W-L%
nhldf = nhldf.drop_duplicates(subset="Metropolitan area").reset_index()
nhldf = nhldf.drop(columns="index")
    
nba_df = nba_df[nba_df["year"] == 2018]
nba_df["team"] = nba_df["team"].apply(lambda x: re.sub(r"(\*)*\s\(\d+\)", "", x))
nba_df["team"] = nba_df["team"].apply(lambda x: x.split(" ")[-1])
nbadf = pd.merge(cities, nba_df, left_on="NBA", right_on="team")
nbadf = nbadf.rename(columns={"W/L%": "W-L%"})
nbadf = nbadf[["Metropolitan area", "Population (2016 est.)[8]", "NBA", "team", "W", "L", "W-L%"]]
nbadf["Population (2016 est.)[8]"] = nbadf["Population (2016 est.)[8]"].astype("float")
nbadf["W-L%"] = nbadf["W-L%"].astype("float")
nbadf.loc[nbadf["Metropolitan area"] == "New York City", "W-L%"] = 0.34750000000000003 
nbadf.loc[nbadf["Metropolitan area"] == "Los Angeles", "W-L%"] = 0.46950000000000003 
nbadf = nbadf.drop_duplicates(subset="Metropolitan area").reset_index()
nbadf = nbadf.drop(columns="index")


mlb_df = mlb_df[mlb_df["year"] == 2018]
mlb_df["team"] = mlb_df["team"].apply(lambda x: x.split(" ")[-1])
mlbdf = pd.merge(cities, mlb_df, left_on="MLB", right_on="team")
mlbdf = mlbdf[["Metropolitan area", "Population (2016 est.)[8]", "MLB", "team", "W", "L", "W-L%"]]
mlbdf["Population (2016 est.)[8]"] = mlbdf["Population (2016 est.)[8]"].astype("float")
mlbdf["W-L%"] = mlbdf["W-L%"].astype("float")
mlbdf.loc[mlbdf["Metropolitan area"] == "New York City", "W-L%"] = 0.546 
mlbdf.loc[mlbdf["Metropolitan area"] == "Los Angeles", "W-L%"] = 0.5289999999999999 
mlbdf.loc[mlbdf["Metropolitan area"] == "San Francisco Bay Area", "W-L%"] = 0.525 
mlbdf.loc[mlbdf["Metropolitan area"] == "Chicago", "W-L%"] = 0.482769 
mlbdf.loc[mlbdf["Metropolitan area"] == "Boston", "W-L%"] = 0.666667 
mlbdf = mlbdf.drop_duplicates(subset="Metropolitan area").reset_index()
mlbdf = mlbdf.drop(columns="index")


nfl_df = nfl_df[nfl_df["year"] == 2018]
nfl_df["team"] = nfl_df["team"].apply(lambda x: re.sub(r"(\*|\+)", "", x))
nfl_df["team"] = nfl_df["team"].apply(lambda x: x.split(" ")[-1])
nfldf = pd.merge(cities, nfl_df, left_on="NFL", right_on="team")
nfldf = nfldf[["Metropolitan area", "Population (2016 est.)[8]", "NFL", "team", "W", "L", "W-L%"]]
nfldf["Population (2016 est.)[8]"] = nfldf["Population (2016 est.)[8]"].astype("float")
nfldf["W-L%"] = nfldf["W-L%"].astype("float")
nfldf.loc[nfldf["Metropolitan area"] == "New York City", "W-L%"] = 0.2815 
nfldf.loc[nfldf["Metropolitan area"] == "Los Angeles", "W-L%"] = 0.7815 
nfldf.loc[nfldf["Metropolitan area"] == "San Francisco Bay Area", "W-L%"] = 0.25 
nfldf = nfldf.drop_duplicates(subset="Metropolitan area").reset_index()
nfldf = nfldf.drop(columns="index")
nbadf

,Metropolitan area,Population (2016 est.)[8],NBA,team,W,L,W-L%
0,New York City,20153634.0,Knicks,Knicks,29,53,0.3475
1,Los Angeles,13310447.0,Lakers,Lakers,35,47,0.4695
2,San Francisco Bay Area,6657982.0,Warriors,Warriors,58,24,0.7070
3,Chicago,9512999.0,Bulls,Bulls,27,55,0.3290
4,Dallas–Fort Worth,7233323.0,Mavericks,Mavericks,24,58,0.2930
5,"Washington, D.C.",6131977.0,Wizards,Wizards,43,39,0.5240
6,Philadelphia,6070500.0,76ers,76ers,52,30,0.6340
7,Boston,4794447.0,Celtics,Celtics,55,27,0.6710
8,Minneapolis–Saint Paul,3551036.0,Timberwolves,Timberwolves,47,35,0.5730
9,Denver,2853077.0,Nuggets,Nuggets,46,36,0.5610


In [6]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nhl_correlation(): 
    # YOUR CODE HERE
    #raise NotImplementedError()
    
    population_by_region = nhldf["Population (2016 est.)[8]"] # pass in metropolitan area population from cities
    win_loss_by_region = nhldf["W-L%"] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    cor = stats.pearsonr(population_by_region, win_loss_by_region)
    return cor[0]

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [15]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nba_correlation():
    # YOUR CODE HERE
    #raise NotImplementedError()
    
    population_by_region = nbadf["Population (2016 est.)[8]"] # pass in metropolitan area population from cities
    win_loss_by_region = nbadf["W-L%"] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    cor = stats.pearsonr(population_by_region, win_loss_by_region)
    return cor[0]

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [16]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def mlb_correlation(): 
    # YOUR CODE HERE
    #raise NotImplementedError()
    
    population_by_region = mlbdf["Population (2016 est.)[8]"] # pass in metropolitan area population from cities
    win_loss_by_region = mlbdf["W-L%"] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    cor = stats.pearsonr(population_by_region, win_loss_by_region)
    return cor[0]

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [5]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nfl_correlation(): 
    # YOUR CODE HERE
    #raise NotImplementedError()
    
    population_by_region = nfldf["Population (2016 est.)[8]"] # pass in metropolitan area population from cities
    win_loss_by_region = nfldf["W-L%"] # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    cor = stats.pearsonr(population_by_region, win_loss_by_region)
    return cor[0]

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def sports_team_performance():
    # YOUR CODE HERE
    raise NotImplementedError()
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values